# Import library yang dibutuhkan

In [1]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

In [2]:
!pip install keras

In [3]:
!pip install -U tfx

In [ ]:
import os
import sys
import transformers
from typing import Text

from absl import logging
from tfx.orchestration import metadata, pipeline
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner


# Set variabel dan Menjalankan ML Pipeline

In [4]:
PIPELINE_NAME = "diabetes-pipeline"

# pipeline inputs
DATA_ROOT = "data"
TRANSFORM_MODULE_FILE = "modules/transform.py"
TRAINER_MODULE_FILE = "modules/trainer.py"
# requirement_file = os.path.join(root, "requirements.txt")

# pipeline outputs

OUTPUT_BASE = "output"
serving_model_dir = os.path.join(OUTPUT_BASE, 'serving_model')
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")

def init_local_pipeline(
    components, pipeline_root: Text
) -> pipeline.Pipeline:

    logging.info(f"Pipeline root set to: {pipeline_root}")
    beam_args = [
        "--direct_running_mode=multi_processing"
        # 0 auto-detect based on on the number of CPUs available
        # during execution time.
        "----direct_num_workers=0"
    ]

    return pipeline.Pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=pipeline_root,
        components=components,
        enable_cache=True,
        metadata_connection_config=metadata.sqlite_metadata_connection_config(
            metadata_path
        ),
        eam_pipeline_args=beam_args
    )

# Menjalankan ML Pipeline
if __name__ == "__main__":
    logging.set_verbosity(logging.INFO)

    from modules.components import init_components

    components = init_components(
        DATA_ROOT,
        training_module=TRAINER_MODULE_FILE,
        transform_module=TRANSFORM_MODULE_FILE,
        training_steps=5000,
        eval_steps=1000,
        serving_model_dir=serving_model_dir,
    )

    pipeline = init_local_pipeline(components, pipeline_root)
    BeamDagRunner().run(pipeline=pipeline)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Pipeline root set to: output/diabetes-pipeline
INFO:absl:Generating ephemeral wheel package for '/content/modules/transform.py' (including modules: ['transform', 'components', 'trainer']).
INFO:absl:User module package has hash fingerprint version c03c440314bd16a4f3ff276e26c7f980189773eebc103f436030fb8ba5817211.
INFO:absl:Executing: ['/usr/bin/python3', '/tmp/tmp6xjqgn2j/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmp7znbs2ib', '--dist-dir', '/tmp/tmp2rlwwk3h']
INFO:absl:Successfully built user code wheel dist

INFO:absl:Node CsvExampleGen depends on [].
INFO:absl:Node CsvExampleGen is scheduled.
INFO:absl:Node Latest_blessed_model_resolver depends on [].
INFO:absl:Node Latest_blessed_model_resolver is scheduled.
INFO:absl:Node StatisticsGen depends on ['Run[CsvExampleGen]'].
INFO:absl:Node StatisticsGen is scheduled.
INFO:absl:Node SchemaGen depends on ['Run[StatisticsGen]'].
INFO:absl:Node SchemaGen is scheduled.
INFO:absl:Node ExampleValidator depends on ['Run[SchemaGen]', 'Run[StatisticsGen]'].
INFO:absl:Node ExampleValidator is scheduled.
INFO:absl:Node Transform depends on ['Run[CsvExampleGen]', 'Run[SchemaGen]'].
INFO:absl:Node Transform is scheduled.
INFO:absl:Node Trainer depends on ['Run[SchemaGen]', 'Run[Transform]'].
INFO:absl:Node Trainer is scheduled.
INFO:absl:Node Evaluator depends on ['Run[CsvExampleGen]', 'Run[Latest_blessed_model_resolver]', 'Run[Trainer]'].
INFO:absl:Node Evaluator is scheduled.
INFO:absl:Node Pusher depends on ['Run[Evaluator]', 'Run[Trainer]'].
INFO:absl

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 gender_xf (InputLayer)      [(None, 4)]                  0         []                            
                                                                                                  
 smoking_history_xf (InputL  [(None, 7)]                  0         []                            
 ayer)                                                                                            
                                                                                                  
 age_xf (InputLayer)         [(None, 1)]                  0         []                            
                                                                                                  
 hypertension_xf (InputLaye  [(None, 1)]                  0         []                        

INFO:absl:Feature HbA1c_level has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature age has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature blood_glucose_level has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature bmi has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature diabetes has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature gender has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature heart_disease has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature hypertension has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature smoking_history has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Function `serve_tf_examples_fn` contains input name(s) 1453662, 1453672, resource with unsupported characters which will be renamed to transform_features_layer_1453662, transform_features_layer_1453672, model_dense_3_biasadd_

# Bagian Baru